In [1]:
from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn, rnn
from time import time
import mxnet as mx
import d2lzh as d2l
import random
import zipfile
import math
import time
import sys 

# 06. 循环神经⽹络

## 6.5 循环神经网络的简洁实现
本节将使用Gluon来更简洁地实现基于循环神经网络的语言模型。

In [2]:
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

### 6.5.1 定义模型
`Gluon`的`rnn`模块提供了循环神经网络的实现。下面构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层`rnn_layer`，并对权重做初始化。

In [3]:
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens) 
rnn_layer.initialize()

接下来调⽤`rnn_layer`的成员函数`begin_state`来返回初始化的隐藏状态列表。它有⼀个形状为(隐藏层个数, 批量⼤小, 隐藏单元个数)的元素。

In [4]:
batch_size = 2
state = rnn_layer.begin_state(batch_size=batch_size) 
state[0].shape

(1, 2, 256)

与上⼀节中实现的循环神经⽹络不同，这⾥`rnn_layer`的输⼊形状为`(时间步数、批量⼤小、输⼊个数)`。其中输⼊个数即one-hot向量⻓度(词典⼤小)。此外，`rnn_layer`作为`Gluon`的`rnn.RNN`实例，在前向计算后会分别返回输出和隐藏状态，其中输出指的是隐藏层在各个时间步上计算并输出的隐藏状态，它们通常作为后续输出层的输⼊。需要强调的是，该`输出`本⾝并不涉及输出层计算，形状为`(时间步数、批量⼤小、隐藏单元个数)`。而`rnn.RNN`实例在前向计算返回的隐藏状态指的是隐藏层在最后时间步的可⽤于初始化下⼀时间步的隐藏状态：当隐藏层有多层时，每⼀层的隐藏状态都会记录在该变量中；对于像⻓短期记忆这样的循环神经⽹络，该变量还会包含其他信息。

In [5]:
num_steps = 35
X = nd.random.uniform(shape=(num_steps, batch_size, vocab_size)) 
Y, state_new = rnn_layer(X, state) 
Y.shape, len(state_new), state_new[0].shape

((35, 2, 256), 1, (1, 2, 256))

接下来我们继承`Block`类来定义⼀个完整的RNN。它⾸先将输⼊数据使⽤one-hot向量表⽰后输⼊到`rnn_layer`中，然后使⽤全连接输出层得到输出。输出个数等于词典⼤小`vocab_size`。

In [6]:
# 本类已保存在d2lzh包中⽅便以后使⽤
class RNNModel(nn.Block):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs) 
        self.rnn = rnn_layer 
        self.vocab_size = vocab_size 
        self.dense = nn.Dense(vocab_size)

    def forward(self, inputs, state):
        # 将输⼊转置成(num_steps, batch_size)后获取one-hot向量表⽰ 
        X = nd.one_hot(inputs.T, self.vocab_size) 
        Y, state = self.rnn(X, state) 
        # 全连接层会⾸先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出 
        # 形状为(num_steps * batch_size, vocab_size) 
        output = self.dense(Y.reshape((-1, Y.shape[-1]))) 
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

### 6.5.2 训练模型
同上⼀节⼀样，下⾯定义⼀个预测函数。这⾥的实现区别在于前向计算和初始化隐藏状态的函数接口。

In [7]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char, char_to_idx):
    # 使⽤model的成员函数来初始化隐藏状态 
    state = model.begin_state(batch_size=1, ctx=ctx) 
    output = [char_to_idx[prefix[0]]] 
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1)) 
        (Y, state) = model(X, state) # 前向计算不需要传⼊模型参数 
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]]) 
        else:
            output.append(int(Y.argmax(axis=1).asscalar())) 
    return ''.join([idx_to_char[i] for i in output])

让我们使用权重为随机值的模型来预测一次。

In [8]:
# ctx = d2l.try_gpu()
ctx = mx.cpu()
model = RNNModel(rnn_layer, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 10, model, vocab_size, ctx, idx_to_char, char_to_idx)

'分开包发茶扫香大开哼水蝶'

接下来实现训练函数。算法同上⼀节的⼀样，但这⾥只使⽤了相邻采样来读取数据。

In [9]:
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx,
    num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0, 'wd': 0})
    # 迭代
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # 梯度裁剪
            params = [p.data() for p in model.collect_params().values()]
            d2l.grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)  # 因为已经误差取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(prefix, pred_len, model, vocab_size, ctx, idx_to_char, char_to_idx))

使⽤和上⼀节实验中⼀样的超参数来训练模型。

In [10]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开'] 
train_and_predict_rnn_gluon(
    model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
    num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 86.755402, time 1.28 sec
 - 分开 我想我的爱写在西元的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
 - 不分开 我想你这生写在西元的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
epoch 100, perplexity 13.590316, time 1.27 sec
 - 分开 娘子我 它不我 说你我的手是  说成了我 我有多有   没有你烦我有多 恼多 我不要再想 我不能再
 - 不分开  我有了界里很 我只念的白 在两银够不够 景色入秋 我该了定我  我的着界 我的完空 我 抛   
epoch 150, perplexity 4.660435, time 1.27 sec
 - 分开 娘子我 谁地神人我 泪双样 分沉 娘子安酒斑鸠 平攻到我们都睡着 我想就这样牵着你的手不放开 娘可
 - 不分开 想要和这 一水晶动 在面的梦 全吹放头发动 景上入秋 漫天黄沙截棍 塞北正气护人多 牧草有没到 你
epoch 200, perplexity 2.465326, time 1.27 sec
 - 分开 她养的黑猫 让我开始乡相信命运 感谢地心引力 让我碰说你 漂亮的让我疯狂的可爱女人 坏坏的让我疯狂
 - 不分开 就是的我爱你你翅膀喘怎 一子我欠见有你 一壶好酒 再来一碗热粥 配上的客栈人多 牧草有没有 我马儿
epoch 250, perplexity 1.742482, time 1.28 sec
 - 分开 问养我 谁在神枪手 巫师 他念念依 快杰好球 怎红出空 没有不同 你不懂 连不知珍重 也有苦依 全
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过我一个功菸飞 迎著了这术馆 让我碰大你 漂亮


## 6.6 通过时间反向传播
如果读者做了上⼀节的练习，就会发现，如果不裁剪梯度，模型将⽆法正常训练。为了深刻理解这⼀现象，本节将介绍循环神经⽹络中梯度的计算和存储⽅法，即`通过时间反向传播`(backpropagation through time)。

我们在`正向传播、反向传播和计算图`⼀节中介绍了神经⽹络中梯度计算与存储的⼀般思路，并强调正向传播和反向传播相互依赖。正向传播在循环神经⽹络中⽐较直观，而通过时间反向传播其实是反向传播在循环神经⽹络中的具体应⽤。我们需要将循环神经⽹络按时间步展开，从而得到模型变量和参数之间的依赖关系，并依据链式法则应⽤反向传播计算并存储梯度。

### 6.6.1 定义模型
简单起见，我们考虑一个无偏差项的RNN，且激活函数为恒等映射($\phi(x)=x$)。设时间步$t$的输入为单样本$\boldsymbol{x}_t \in \mathbb{R}^d$，标签为$y_t$，那么隐藏状态$\boldsymbol{h}_t \in \mathbb{R}^h$的计算表达式为
+ $\boldsymbol{h}_t = \boldsymbol{W}_{hx} \boldsymbol{x}_t + \boldsymbol{W}_{hh} \boldsymbol{h}_{t-1}$

其中$\boldsymbol{W}_{hx} \in \mathbb{R}^{h \times d}$和$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$是隐藏层权重参数。

设输出层权重参数$\boldsymbol{W}_{qh} \in \mathbb{R}^{q \times h}$，时间步$t$的输出层变量$\boldsymbol{o}_t \in \mathbb{R}^q$计算为
+ $\boldsymbol{o}_t = \boldsymbol{W}_{qh} \boldsymbol{h}_{t}$

设时间步$t$的损失为$\ell(\boldsymbol{o}_t, y_t)$。时间步数为$T$的损失函数$L$定义为
+ $\displaystyle L = \frac{1}{T} \sum_{t=1}^T \ell (\boldsymbol{o}_t, y_t)$

我们将$L$称为有关给定时间步的数据样本的目标函数，并在本节后续讨论中简称为目标函数。

### 6.6.2 模型计算图
为了可视化循环神经网络中模型变量和参数在计算中的依赖关系，我们可以绘制模型计算图。例如，时间步3的隐藏状态$\boldsymbol{h}_3$的计算依赖模型参数$\boldsymbol{W}_{hx}$、$\boldsymbol{W}_{hh}$、上一时间步隐藏状态$\boldsymbol{h}_2$以及当前时间步输入$\boldsymbol{x}_3$。

<img src="images/06_03.png" style="width:700px;"/>

### 6.6.3 方法
`图6.3`中的模型的参数是$\boldsymbol{W}_{hx}$、$\boldsymbol{W}_{hh}$和$\boldsymbol{W}_{qh}$。与`正向传播、反向传播和计算图`一节中的类似，训练模型通常需要模型参数的梯度$\partial L/\partial \boldsymbol{W}_{hx}$、$\partial L/\partial \boldsymbol{W}_{hh}$和$\partial L/\partial \boldsymbol{W}_{qh}$。根据`图6.3`中的依赖关系，我们可以按照其中箭头所指的反方向依次计算并存储梯度。为了表述方便，我们依然采用`正向传播、反向传播和计算图`一节中表达链式法则的运算符`prod`。

首先，目标函数有关各时间步输出层变量的梯度$\partial L/\partial \boldsymbol{o}_t \in \mathbb{R}^q$很容易计算：
+ $\displaystyle\frac{\partial L}{\partial \boldsymbol{o}_t} = \frac{\partial \ell (\boldsymbol{o}_t, y_t)}{T \cdot \partial \boldsymbol{o}_t}$

下面，我们可以计算目标函数有关模型参数$\boldsymbol{W}_{qh}$的梯度$\partial L/\partial \boldsymbol{W}_{qh} \in \mathbb{R}^{q \times h}$。根据`图6.3`，$L$通过$\boldsymbol{o}_1, \ldots, \boldsymbol{o}_T$依赖$\boldsymbol{W}_{qh}$。依据链式法则，
+ $\displaystyle\frac{\partial L}{\partial \boldsymbol{W}_{qh}} = \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{o}_t}, \frac{\partial \boldsymbol{o}_t}{\partial \boldsymbol{W}_{qh}}\right) = \sum_{t=1}^T \frac{\partial L}{\partial \boldsymbol{o}_t} \boldsymbol{h}_t^\top$

其次，我们注意到隐藏状态之间也存在依赖关系。在`图6.3`中，$L$只通过$\boldsymbol{o}_T$依赖最终时间步$T$的隐藏状态$\boldsymbol{h}_T$。因此，我们先计算目标函数有关最终时间步隐藏状态的梯度$\partial L/\partial \boldsymbol{h}_T \in \mathbb{R}^h$。依据链式法则，我们得到
+ $\displaystyle\frac{\partial L}{\partial \boldsymbol{h}_T} = \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{o}_T}, \frac{\partial \boldsymbol{o}_T}{\partial \boldsymbol{h}_T} \right) = \boldsymbol{W}_{qh}^\top \frac{\partial L}{\partial \boldsymbol{o}_T}$

接下来对于时间步$t < T$，在`图6.3`中，$L$通过$\boldsymbol{h}_{t+1}$和$\boldsymbol{o}_t$依赖$\boldsymbol{h}_t$。依据链式法则，目标函数有关时间步$t < T$的隐藏状态的梯度$\partial L/\partial \boldsymbol{h}_t \in \mathbb{R}^h$需要按照时间步从大到小依次计算：
+ $\displaystyle\frac{\partial L}{\partial \boldsymbol{h}_t} = \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{h}_{t+1}}, \frac{\partial \boldsymbol{h}_{t+1}}{\partial \boldsymbol{h}_t} \right) + \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{o}_t}, \frac{\partial \boldsymbol{o}_t}{\partial \boldsymbol{h}_t} \right) = \boldsymbol{W}_{hh}^\top \frac{\partial L}{\partial \boldsymbol{h}_{t+1}} + \boldsymbol{W}_{qh}^\top \frac{\partial L}{\partial \boldsymbol{o}_t}$

将上面的递归公式展开，对任意时间步$1 \leq t \leq T$，我们可以得到目标函数有关隐藏状态梯度的通项公式
+ $\displaystyle\frac{\partial L}{\partial \boldsymbol{h}_t} = \sum_{i=t}^T {\left(\boldsymbol{W}_{hh}^\top\right)}^{T-i} \boldsymbol{W}_{qh}^\top \frac{\partial L}{\partial \boldsymbol{o}_{T+t-i}}$

由上式中的指数项可见，当时间步数$T$较大或者时间步$t$较小时，目标函数有关隐藏状态的梯度较容易出现衰减和爆炸。这也会影响其他包含$\partial L / \partial \boldsymbol{h}_t$项的梯度，例如隐藏层中模型参数的梯度$\partial L / \partial \boldsymbol{W}_{hx} \in \mathbb{R}^{h \times d}$和$\partial L / \partial \boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$。

在`图6.3`中，$L$通过$\boldsymbol{h}_1, \ldots, \boldsymbol{h}_T$依赖这些模型参数。依据链式法则，我们有
$$\displaystyle\begin{aligned} \frac{\partial L}{\partial \boldsymbol{W}_{hx}} &= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{h}_t}, \frac{\partial \boldsymbol{h}_t}{\partial \boldsymbol{W}_{hx}}\right) = \sum_{t=1}^T \frac{\partial L}{\partial \boldsymbol{h}_t} \boldsymbol{x}_t^\top,\\ \frac{\partial L}{\partial \boldsymbol{W}_{hh}} &= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \boldsymbol{h}_t}, \frac{\partial \boldsymbol{h}_t}{\partial \boldsymbol{W}_{hh}}\right) = \sum_{t=1}^T \frac{\partial L}{\partial \boldsymbol{h}_t} \boldsymbol{h}_{t-1}^\top. \end{aligned} $$

我们已在`正向传播、反向传播和计算图`一节里解释过，每次迭代中，我们在依次计算完以上各个梯度后，会将它们存储起来，从而避免重复计算。例如，由于隐藏状态梯度$\partial L/\partial \boldsymbol{h}_t$被计算和存储，之后的模型参数梯度$\partial L/\partial \boldsymbol{W}_{hx}$和$\partial L/\partial \boldsymbol{W}_{hh}$的计算可以直接读取$\partial L/\partial \boldsymbol{h}_t$的值，而无须重复计算它们。此外，反向传播中的梯度计算可能会依赖变量的当前值。它们正是通过正向传播计算出来的。举例来说，参数梯度$\partial L/\partial \boldsymbol{W}_{hh}$的计算需要依赖隐藏状态在时间步$t = 0, \ldots, T-1$的当前值$\boldsymbol{h}_t$($\boldsymbol{h}_0$是初始化得到的)。这些值是通过从输入层到输出层的正向传播计算并存储得到的。

## 6.7 ⻔控循环单元(GRU)
上一节介绍了循环神经网络中的梯度计算方法。当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较难捕捉时间序列中时间步距离较大的依赖关系。

门控循环神经网络(gated recurrent neural network)的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。门控循环单元(gated recurrent unit，GRU)是一种常用的门控循环神经网络。另一种常用的门控循环神经网络则将在下一节中介绍。

### 6.7.1 门控循环单元
下面将介绍门控循环单元的设计。它引入了`重置门`(reset gate)和`更新门`(update gate)的概念，从而修改了循环神经网络中隐藏状态的计算方式。

##### 重置门和更新门
如`图6.4`所示，门控循环单元中的重置门和更新门的输入均为当前时间步输入$\boldsymbol{X}t$与上一时间步隐藏状态$\boldsymbol{H}{t-1}$，输出由激活函数为`sigmoid`函数的全连接层计算得到。

<img src="images/06_04.png" style="width:550px;"/>

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$(样本数为$n$，输入个数为$d$)和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。重置门$\boldsymbol{R}_t \in \mathbb{R}^{n \times h}$和更新门$\boldsymbol{Z}_t \in \mathbb{R}^{n \times h}$的计算如下：
$$\begin{aligned} \boldsymbol{R}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xr} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hr} + \boldsymbol{b}_r),\\ \boldsymbol{Z}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xz} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hz} + \boldsymbol{b}_z), \end{aligned} $$

其中$\boldsymbol{W}_{xr}, \boldsymbol{W}_{xz} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hr}, \boldsymbol{W}_{hz} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_r, \boldsymbol{b}_z \in \mathbb{R}^{1 \times h}$是偏差参数。`sigmoid`函数将重置门$\boldsymbol{R}_t$和更新门$\boldsymbol{Z}_t$中每个元素的值域都是$[0, 1]$。

##### 候选隐藏状态
接下来，门控循环单元将计算候选隐藏状态来辅助稍后的隐藏状态计算。如`图6.5`所示，我们将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法(符号为$\odot$)。如果重置门中元素值接近0，那么意味着重置对应隐藏状态元素为0，即丢弃上一时间步的隐藏状态。如果元素值接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数`tanh`的全连接层计算出候选隐藏状态，其所有元素的值域为$[-1, 1]$。

<img src="images/06_05.png" style="width:550px;"/>

具体来说，时间步$t$的候选隐藏状态$\tilde{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$的计算为
+ $\tilde{\boldsymbol{H}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \left(\boldsymbol{R}_t \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{hh} + \boldsymbol{b}_h)$

其中$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$是偏差参数。从上面这个公式可以看出，重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。

##### 隐藏状态
最后，时间步$t$的隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的计算使用当前时间步的更新门$\boldsymbol{Z}_t$来对上一时间步的隐藏状态$\boldsymbol{H}_{t-1}$和当前时间步的候选隐藏状态$\tilde{\boldsymbol{H}}_t$做组合：
+ $\boldsymbol{H}_t = \boldsymbol{Z}_t \odot \boldsymbol{H}_{t-1} + (1 - \boldsymbol{Z}_t) \odot \tilde{\boldsymbol{H}}_t$

<img src="images/06_06.png" style="width:550px;"/>

值得注意的是，更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如`图6.6`所示。假设更新门在时间步$t'$到$t$($t' < t$)之间一直近似1。那么，在时间步$t'$到$t$之间的输入信息几乎没有流入时间步$t$的隐藏状态$\boldsymbol{H}_t$。实际上，这可以看作是较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存并传递至当前时间步$t$。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

我们对门控循环单元的设计稍作总结：
+ 重置门有助于捕捉时间序列里短期的依赖关系
+ 更新门有助于捕捉时间序列里长期的依赖关系


### 6.7.2 读取数据集
为了实现并展示门控循环单元，下面依然使用周杰伦歌词数据集来训练模型作词。

In [11]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

### 6.7.3 从零开始实现
我们先介绍如何从零开始实现门控循环单元。

##### 初始化模型参数
下面的代码对模型参数进行初始化。超参数`num_hiddens`定义了隐藏单元的个数。

In [12]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# ctx = d2l.try_gpu()
ctx = mx.cpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)), _one((num_hiddens, num_hiddens)), nd.zeros(num_hiddens, ctx=ctx))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

##### 定义模型
下面的代码定义隐藏状态初始化函数`init_gru_state`。同`循环神经网络的从零开始实现`中定义的`init_rnn_state`函数一样，它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。

In [13]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

下面根据门控循环单元的计算表达式定义模型。

In [14]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = [] 
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z) 
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r) 
        H_tilda = nd.tanh(nd.dot(X, W_xh) + nd.dot(R * H, W_hh) + b_h) 
        H = Z * H + (1 - Z) * H_tilda 
        Y = nd.dot(H, W_hq) + b_q 
        outputs.append(Y)
    return outputs, (H,)

##### 训练模型并创作歌词
我们在训练模型时只使用相邻采样。设置好超参数后，我们将训练模型并根据前缀`分开`和`不分开`分别创作长度为50个字符的一段歌词。

In [15]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(
    gru, get_params, init_gru_state, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
    False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 150.668060, time 3.19 sec
 - 分开 我想你的让我不想想想想想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你
 - 不分开 我想你的让我不想想想想想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你想你
epoch 80, perplexity 32.527277, time 3.23 sec
 - 分开 我想要你的微笑 一定后起 你想就这样 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 
 - 不分开 不知不觉 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch 120, perplexity 5.937680, time 3.16 sec
 - 分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 一壶好
 - 不分开 不知再这样打我妈妈 我说的话你笑听听 不要再这样打我妈妈 难道你手不会痛吗 我想要这样牵著你 想想
epoch 160, perplexity 1.799869, time 3.15 sec
 - 分开 一直我 泪你是一手 后知后觉 我该好好生活 我该好好生活 静静悄悄默默离开 陷入了危险边缘Baby
 - 不分开不要 不想再你 我想要这样 我的你以 没托了没怎 我有多烦 你已怎么在我想开不开 我给的这色 你后开


### 6.7.4 简洁实现
在`Gluon`中我们直接调⽤`rnn`模块中的`GRU`类即可。

In [16]:
gru_layer = rnn.GRU(num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size) 
d2l.train_and_predict_rnn_gluon(
    model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, num_epochs, num_steps, 
    lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 154.700082, time 2.96 sec
 - 分开 我想你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 
 - 不分开 我想你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 你不你的让我 
epoch 80, perplexity 33.612999, time 3.08 sec
 - 分开 一直我 别子我 别你的话笑 一场寄人 不知不觉 我不要这样 我不要再想你 我不要再想你 我不要再想
 - 不分开 爱你在我不多 想要你的爱笑 让我想想你的爱笑 一定在美不 你想要你的爱笑 像思的美不面 想要你的爱
epoch 120, perplexity 5.120571, time 3.08 sec
 - 分开 我想要这样 我不要再想你 爱情 是你开 我想要你 我想要这样打我妈妈 不懂 你想很久了吧? 我的认
 - 不分开 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活
epoch 160, perplexity 1.465803, time 3.01 sec
 - 分开的让 感放动 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看远方的星
 - 不分开做  没有你烦不多 难发抖 你怎一句看守 他的让我面无的可爱女人 温柔的让我心疼的可爱女人 透明的让


## 6.8 长短期记忆（LSTM）
长短期记忆(long short-term memory，LSTM)是另一种常用的门控循环神经网络。

### 6.8.1 长短期记忆
`LSTM`中引入了3个门，即输入门(input gate)、遗忘门(forget gate)和输出门(output gate)，以及与隐藏状态形状相同的记忆细胞(某些文献把记忆细胞当成一种特殊的隐藏状态)，从而记录额外的信息。

###### 输入门、遗忘门和输出门
与门控循环单元中的重置门和更新门一样，如`图6.7`所示，长短期记忆的门的输入均为当前时间步输入$\boldsymbol{X}t$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$，输出由激活函数为`sigmoid`函数的全连接层计算得到。如此一来，这3个门元素的值域均为$[0,1]$。

<img src="images/06_07.png" style="width:600px;"/>

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$(样本数为$n$，输入个数为$d$)和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。 时间步$t$的输入门$\boldsymbol{I}_t \in \mathbb{R}^{n \times h}$、遗忘门$\boldsymbol{F}_t \in \mathbb{R}^{n \times h}$和输出门$\boldsymbol{O}_t \in \mathbb{R}^{n \times h}$分别计算如下：
$$\begin{aligned} \boldsymbol{I}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hi} + \boldsymbol{b}_i),\\ \boldsymbol{F}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hf} + \boldsymbol{b}_f),\\ \boldsymbol{O}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{ho} + \boldsymbol{b}_o) \end{aligned} $$

其中的$\boldsymbol{W}_{xi}, \boldsymbol{W}_{xf}, \boldsymbol{W}_{xo} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hi}, \boldsymbol{W}_{hf}, \boldsymbol{W}_{ho} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_i, \boldsymbol{b}_f, \boldsymbol{b}_o \in \mathbb{R}^{1 \times h}$是偏差参数。

##### 候选记忆细胞
接下来，长短期记忆需要计算候选记忆细胞$\tilde{\boldsymbol{C}}_t$。它的计算与上面介绍的3个门类似，但使用了值域在$[-1, 1]$的`tanh`函数作为激活函数。

<img src="images/06_08.png" style="width:600px;"/>

具体来说，时间步$t$的候选记忆细胞$\tilde{\boldsymbol{C}}_t \in \mathbb{R}^{n \times h}$的计算为
$$\tilde{\boldsymbol{C}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hc} + \boldsymbol{b}_c)$$

其中$\boldsymbol{W}_{xc} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hc} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_c \in \mathbb{R}^{1 \times h}$是偏差参数。

##### 记忆细胞
我们可以通过元素值域在$[0, 1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法(符号为$\odot$)来实现的。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：
$$\boldsymbol{C}_t = \boldsymbol{F}_t \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_t \odot \tilde{\boldsymbol{C}}_t$$

如`图6.9`所示，遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$中的信息是否传递到当前时间步，而输入门则控制当前时间步的输入$\boldsymbol{X}_t$通过候选记忆细胞$\tilde{\boldsymbol{C}}_t$如何流入当前时间步的记忆细胞。如果遗忘门一直近似1且输入门一直近似0，过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

<img src="images/06_09.png" style="width:600px;"/>

##### 隐藏状态
有了记忆细胞以后，接下来我们还可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动：
$$\boldsymbol{H}_t = \boldsymbol{O}_t \odot \text{tanh}(\boldsymbol{C}_t).$$

这里的`tanh`函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。`图6.10`展示了长短期记忆中隐藏状态的计算。

<img src="images/06_10.png" style="width:600px;"/>


### 6.8.2 读取数据集
下面我们开始实现并展示长短期记忆。

In [17]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

### 6.8.3 从零开始实现
##### 初始化模型参数
下⾯的代码对模型参数进⾏初始化。超参数`num_hiddens`定义了隐藏单元的个数。

In [18]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# ctx = d2l.try_gpu()
ctx = mx.cpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)), _one((num_hiddens, num_hiddens)), nd.zeros(num_hiddens, ctx=ctx))

    W_xi, W_hi, b_i = _three() # 输⼊⻔参数
    W_xf, W_hf, b_f = _three() # 遗忘⻔参数
    W_xo, W_ho, b_o = _three() # 输出⻔参数
    W_xc, W_hc, b_c = _three() # 候选记忆细胞参数
    # 输出层参数 
    W_hq = _one((num_hiddens, num_outputs)) 
    b_q = nd.zeros(num_outputs, ctx=ctx) 
    # 附上梯度 
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] 
    for param in params:
        param.attach_grad() 
    return params

### 6.8.4 定义模型
在初始化函数中，长短期记忆的隐藏状态需要返回额外的形状为(批量大小, 隐藏单元个数)的值为0的记忆细胞。

In [19]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    return (
        nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), 
        nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

下⾯根据⻓短期记忆的计算表达式定义模型。需要注意的是，只有隐藏状态会传递到输出层，而记忆细胞不参与输出层的计算。

In [20]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

##### 训练模型并创作歌词
我们在训练模型时只使⽤相邻采样。我们将训练模型并根据前缀`分开`和`不分开`分别创作⻓度为50个字符的⼀段歌词。

In [21]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(
    lstm, get_params, init_lstm_state, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, 
    False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 212.155660, time 3.94 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 67.664784, time 4.01 sec
 - 分开 我想你这你 我不要 我不我 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
 - 不分开 我想你这你 我不要 我想你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要
epoch 120, perplexity 15.272441, time 4.03 sec
 - 分开 我想你的生笑 一少 在我不多的白? 我的店 快沉两 三么 在什么 干么么 干么么 干么么 干么么 
 - 不分开 我已我 你你我 我想要这样 我有样这样活 我知你觉 我该了好节秋 后知后觉 我该好好生活 后知后觉
epoch 160, perplexity 3.916873, time 3.89 sec
 - 分开 我想带你的微笑天天想 想要说说 你却我早已力 不要我没有你有一场  有有你的爸笑听听的家 你说说说
 - 不分开你已知  我要你的微笑 我 想你和看堡著 想这样没样忧我妈妈 我想你你已经很听 别要我 说你眼睛看着


### 6.8.5 简洁实现
在`Gluon`中我们可以直接调⽤`rnn`模块中的`LSTM`类。

In [22]:
lstm_layer = rnn.LSTM(num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size) 
d2l.train_and_predict_rnn_gluon(
    model, num_hiddens, vocab_size, ctx, corpus_indices, idx_to_char, char_to_idx, num_epochs, num_steps, 
    lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 221.509106, time 4.09 sec
 - 分开 我不不我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我
 - 不分开 我不不我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我
epoch 80, perplexity 66.716124, time 4.34 sec
 - 分开 你想你的你 我不要这你 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我
 - 不分开 我想你你的你 我想你这你 我不要这你 我不要你 我不要这生 我不要这生 我不要这生 我不要这生 我
epoch 120, perplexity 14.600561, time 4.20 sec
 - 分开 你在我 你子我 一九我 一话的碗  一上在 在谁了 有有了 装片么 一片空 停不么 一片空 一片么
 - 不分开你 我想要这样笑 我想要你你不笑 不要 你想我 我不要再生活 快知后觉 你想了这生活 我知好好生活 
epoch 160, perplexity 4.529191, time 4.18 sec
 - 分开 你小的话不么 我学带  你常来的手 快后在够只 静静这直落 这底拽什么 不爽就斗走 有种拽篮驳 有
 - 不分开 你已你的生笑 不么  又跟我的太活 一后在后后 一直到 在都了空的我 就知会觉 我该好这生活 后知


## 6.9 深度循环神经⽹络
本章到⽬前为⽌介绍的循环神经⽹络只有⼀个单向的隐藏层，在深度学习应⽤⾥，我们通常会⽤到含有多个隐藏层的循环神经⽹络，也称作`深度循环神经⽹络`。`图6.11`演⽰了⼀个有`L`个隐藏层的深度循环神经⽹络，每个隐藏状态不断传递⾄当前层的下⼀时间步和当前时间步的下⼀层。

<img src="images/06_11.png" style="width:400px;"/>

具体来说，在时间步$t$里，设小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$(样本数为$n$，输入个数为$d$)，第$l$隐藏层($l=1,\ldots,L$)的隐藏状态为$\boldsymbol{H}_t^{(l)} \in \mathbb{R}^{n \times h}$（隐藏单元个数为$h$），输出层变量为$\boldsymbol{O}_t \in \mathbb{R}^{n \times q}$(输出个数为$q$)，且隐藏层的激活函数为$\phi$。第1隐藏层的隐藏状态和之前的计算一样：
+ $\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)} + \boldsymbol{b}_h^{(1)})$

其中权重$\boldsymbol{W}_{xh}^{(1)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(1)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(1)} \in \mathbb{R}^{1 \times h}$分别为第1隐藏层的模型参数。

当$1 < l \leq L$时，第$l$隐藏层的隐藏状态的表达式为：
+ $\boldsymbol{H}_t^{(l)} = \phi(\boldsymbol{H}_t^{(l-1)} \boldsymbol{W}_{xh}^{(l)} + \boldsymbol{H}_{t-1}^{(l)} \boldsymbol{W}_{hh}^{(l)} + \boldsymbol{b}_h^{(l)})$

其中权重$\boldsymbol{W}_{xh}^{(l)} \in \mathbb{R}^{h \times h}$、$\boldsymbol{W}_{hh}^{(l)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(l)} \in \mathbb{R}^{1 \times h}$分别为第$l$隐藏层的模型参数。

最终，输出层的输出只需基于第$L$隐藏层的隐藏状态：
+ $\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q$

其中权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$为输出层的模型参数。

同多层感知机一样，隐藏层个数$L$和隐藏单元个数$h$都是超参数。此外，如果将隐藏状态的计算换成门控循环单元或者长短期记忆的计算，我们可以得到深度门控循环神经网络。

## 6.10 双向循环神经网络
之前介绍的循环神经网络模型都是假设当前时间步是由前面的较早时间步的序列决定的，因此它们都将信息通过隐藏状态从前往后传递。有时候，当前时间步也可能由后面时间步决定。例如，当我们写下一个句子时，可能会根据句子后面的词来修改句子前面的用词。双向循环神经网络通过增加从后往前传递信息的隐藏层来更灵活地处理这类信息。`图6.12`演示了一个含单隐藏层的双向循环神经网络的架构。

<img src="images/06_12.png" style="width:350px;"/>

给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$(样本数为$n$，输入个数为$d$)和隐藏层激活函数为$\phi$。在双向循环神经网络的架构中，设该时间步正向隐藏状态为$\overrightarrow{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$(正向隐藏单元个数为$h$)， 反向隐藏状态为$\overleftarrow{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$(反向隐藏单元个数为$h$)。我们可以分别计算正向隐藏状态和反向隐藏状态：
$$\displaystyle\begin{aligned} \overrightarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(f)} + \overrightarrow{\boldsymbol{H}}_{t-1} \boldsymbol{W}_{hh}^{(f)} + \boldsymbol{b}_h^{(f)}),\\ \overleftarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(b)} + \overleftarrow{\boldsymbol{H}}_{t+1} \boldsymbol{W}_{hh}^{(b)} + \boldsymbol{b}_h^{(b)}) \end{aligned} $$

其中权重$\boldsymbol{W}_{xh}^{(f)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(f)} \in \mathbb{R}^{h \times h}$、$\boldsymbol{W}_{xh}^{(b)} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh}^{(b)} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h^{(f)} \in \mathbb{R}^{1 \times h}$、$\boldsymbol{b}_h^{(b)} \in \mathbb{R}^{1 \times h}$均为模型参数。

然后我们连结两个方向的隐藏状态$\overrightarrow{\boldsymbol{H}}_t$和$\overleftarrow{\boldsymbol{H}}_t$来得到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times 2h}$，并将其输入到输出层。输出层计算输出$\boldsymbol{O}_t \in \mathbb{R}^{n \times q}$(输出个数为$q$)：
+ $\boldsymbol{O}_t = \boldsymbol{H}t \boldsymbol{W}_{hq} + \boldsymbol{b}_q$

其中权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{2h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$为输出层的模型参数。不同方向上的隐藏单元个数也可以不同。